In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

In [2]:
def parseInput(line):
    fields = line.split('|')
    return Row(user_id =int(fields[0]), age=int(fields[1]), gender=fields[2], occupation=fields[3], zip=fields[4])

In [ ]:
if __name__ == "__main__":
    # Create a SparkSession
    spark = SparkSession.builder.appName("MonogDBIntegration").getOrCreate()
    # Building RDD on top of users data file
    lines = spark.sparkContext.textFile("hdfs:///user/maria_dev/mongodb/movies.user")
    # Creating new RDD by passing the parsers functions 
    users = lines.map(parseInput)
    # Converting RDD into a Data Frame
    usersDataset = spark.createDataFrame(users)
    # writing the data into mongo DB
    usersDataset.write.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://127.0.0.1/moviesdata.users")\
    .mode("append").save()